In [7]:
import pandas as pd
import numpy as np
import datetime
from Trax.Cloud.Services.Connector.Logger import LoggerInitializer
from Trax.Utils.Conf.Configuration import Config
from Trax.Data.Projects.Connector import ProjectConnector
from Trax.Cloud.Services.Connector.Keys import DbUsers

LoggerInitializer.init('Diageo Accuracy Test')
Config.set_access_mode(Config.CROSS_CLOUD_ACCESS_MODE)
#Config.set_env_and_cloud(Config.PROD, Config.AWS)
Config.set_env_and_cloud(Config.PROD, Config.GCP)

2021-11-24 15:12:25,407 - Diageo Accuracy Test - 1515 - WARNING - Logger init method is called more than once. Current app:Diageo Accuracy Test, new app: Diageo Accuracy Test 


False

In [8]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [9]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [10]:
from Trax.Cloud.Services.Connector.Factory import BigQueryFactory
try:
    print(bq_client)
except NameError:
    bq_client = BigQueryFactory.get_bigquery_client('trax-ortal-prod')

In [52]:
sql_probes = """ SELECT *
                        from (SELECT timestamp,event_name,session_uid,wave_type,wave_uid,
                                    project_name,probe_id,data_subject,flavor,accuracy_logic_type,
                                    fr.masking_box,fr.majority_product_fk,
                                    fr.voting_result,fr.voting_result_linkage, fr.majority_value,
                                    fr.majority_is_promotion, fr.production_tag_source_group,
                                    fr.production_tag_source, fr.production_tag_identified,
                                    fr.production_product_fk, fr.TP_value, fr.FP_value, fr.FN_value,
                                    fr.INCONCLUSIVE_value, fr.TP_promotion,fr.FP_promotion,fr.FN_promotion,
                                    fr.TP_linkage,fr.FP_linkage,fr.FN_linkage,fr.INCONCLUSIVE_linkage,
                                    dense_rank() over(partition by event_name, project_name, probe_id, 
                                                                   fr.production_tag_source_group order by timestamp desc) as rownum
                                FROM `trax-ortal-prod.raw.factory_accuracy_price`
                                join unnest (additional_event_data) fr
                                WHERE DATE(timestamp) > "2021-11-01")
                    where rownum = 1
                    """
df = bq_client.run_query(sql_probes).to_dataframe()

2021-11-25 10:35:07,509 - Diageo Accuracy Test - 1515 - INFO - Run query with job_id=a8ab0b3e-2473-4c37-948a-cb848e60a39d 
2021-11-25 10:35:18,419 - Diageo Accuracy Test - 1515 - INFO - Query job=a8ab0b3e-2473-4c37-948a-cb848e60a39d finished {'total_mb_billed': 232.783872, 'total_mb_processed': 231.826804}


In [12]:
df.head()

,timestamp,event_name,session_uid,wave_type,wave_uid,project_name,probe_id,data_subject,flavor,accuracy_logic_type,masking_box,majority_product_fk,voting_result,voting_result_linkage,majority_value,majority_is_promotion,production_tag_source_group,production_tag_source,production_tag_identified,production_product_fk,TP_value,FP_value,FN_value,INCONCLUSIVE_value,TP_promotion,FP_promotion,FN_promotion,TP_linkage,FP_linkage,FN_linkage,INCONCLUSIVE_linkage
0,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086182,NaN,inconclusive,inconclusive,NaN,NaN,tag_source_not_relevant,tag_source_not_relevant,0,NaN,0,0,0,1,0,0,0,0,0,0,1
1,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086181,NaN,inconclusive,inconclusive,NaN,NaN,QAT,QAT,1,821.0,0,0,0,1,0,0,0,0,0,0,1
2,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086205,NaN,inconclusive,inconclusive,NaN,NaN,tag_source_not_relevant,tag_source_not_relevant,0,NaN,0,0,0,1,0,0,0,0,0,0,1
3,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086177,NaN,majority,inconclusive,1116.0,1.0,QAT,QAT,1,29.0,1,0,0,0,1,0,0,0,0,0,1
4,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086180,NaN,majority,inconclusive,999.0,1.0,QAT,QAT,1,2142.0,1,0,0,0,1,0,0,0,0,0,1


In [13]:
df.accuracy_logic_type.unique()

array(['internal'], dtype=object)

In [14]:
df.data_subject.unique()

array(['Price'], dtype=object)

In [15]:
df.flavor.unique()

array(['default'], dtype=object)

In [54]:
df.wave_type.unique()

array(['pricing_voting', 'voting'], dtype=object)

In [16]:
t1 = df.groupby(['timestamp','project_name','probe_id','majority_product_fk']).agg({'voting_result':'count'}).reset_index()

In [17]:
t1[t1['voting_result']>1]

,timestamp,project_name,probe_id,majority_product_fk,voting_result
39,2021-11-02 01:04:41.745355+00:00,jnjanz,664187,5510.0,3
41,2021-11-02 01:04:41.745355+00:00,jnjanz,664187,6750.0,11
349,2021-11-03 05:46:28.618424+00:00,danonear,38552,349.0,2
420,2021-11-03 06:24:40.297582+00:00,danonear,40848,410.0,2
441,2021-11-03 06:27:56.353324+00:00,danonear,41332,365.0,2
455,2021-11-03 08:49:42.941671+00:00,diageobenelux,836946,498.0,14
456,2021-11-03 08:49:42.941671+00:00,diageobenelux,836946,524.0,6
458,2021-11-03 08:49:42.941671+00:00,diageobenelux,836946,631.0,2
462,2021-11-03 08:49:42.941671+00:00,diageobenelux,836946,2897.0,3
612,2021-11-04 00:57:56.722732+00:00,nestleus,10896076,67300.0,2


In [31]:
#df[(df['probe_id']==38552) & (df['project_name']=='danonear')&(df['majority_product_fk']==349.0)]
df[(df['probe_id']==38552) & (df['project_name']=='danonear')]

,timestamp,event_name,session_uid,wave_type,wave_uid,project_name,probe_id,data_subject,flavor,accuracy_logic_type,masking_box,majority_product_fk,voting_result,voting_result_linkage,majority_value,majority_is_promotion,production_tag_source_group,production_tag_source,production_tag_identified,production_product_fk,TP_value,FP_value,FN_value,INCONCLUSIVE_value,TP_promotion,FP_promotion,FN_promotion,TP_linkage,FP_linkage,FN_linkage,INCONCLUSIVE_linkage
437831,2021-11-03 05:46:28.618424+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,6b58be2b-2a56-45aa-b533-22b2be91d6b4,voting,f6866b2f-0645-432b-b780-f653d2d07abd,danonear,38552,Price,default,internal,752320,NaN,majority,None,114.0,0.0,QAT,QAT,1,349.0,1,0,0,0,1,0,0,0,0,0,0
437832,2021-11-03 05:46:28.618424+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,6b58be2b-2a56-45aa-b533-22b2be91d6b4,voting,f6866b2f-0645-432b-b780-f653d2d07abd,danonear,38552,Price,default,internal,752329,NaN,majority,None,1525.5,0.0,QAT,QAT,1,353.0,1,0,0,0,1,0,0,0,0,0,0
437833,2021-11-03 05:46:28.618424+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,6b58be2b-2a56-45aa-b533-22b2be91d6b4,voting,f6866b2f-0645-432b-b780-f653d2d07abd,danonear,38552,Price,default,internal,752321,349.0,majority,majority,114.0,0.0,QAT,QAT,1,NaN,1,0,0,0,1,0,0,0,0,1,0
437834,2021-11-03 05:46:28.618424+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,6b58be2b-2a56-45aa-b533-22b2be91d6b4,voting,f6866b2f-0645-432b-b780-f653d2d07abd,danonear,38552,Price,default,internal,752322,349.0,majority,majority,114.0,0.0,QAT,QAT,1,NaN,1,0,0,0,1,0,0,0,0,1,0
437835,2021-11-03 05:46:28.618424+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,6b58be2b-2a56-45aa-b533-22b2be91d6b4,voting,f6866b2f-0645-432b-b780-f653d2d07abd,danonear,38552,Price,default,internal,752325,353.0,majority,majority,1525.5,0.0,QAT,QAT,1,NaN,1,0,0,0,1,0,0,0,0,1,0
437836,2021-11-03 05:46:28.618424+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,6b58be2b-2a56-45aa-b533-22b2be91d6b4,voting,f6866b2f-0645-432b-b780-f653d2d07abd,danonear,38552,Price,default,internal,752324,NaN,majority,None,1525.5,0.0,QAT,QAT,1,NaN,1,0,0,0,1,0,0,1,0,0,0
437837,2021-11-03 05:46:28.618424+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,6b58be2b-2a56-45aa-b533-22b2be91d6b4,voting,f6866b2f-0645-432b-b780-f653d2d07abd,danonear,38552,Price,default,internal,752323,NaN,majority,None,1525.5,0.0,QAT,QAT,1,NaN,1,0,0,0,1,0,0,1,0,0,0
437838,2021-11-03 05:46:28.618424+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,6b58be2b-2a56-45aa-b533-22b2be91d6b4,voting,f6866b2f-0645-432b-b780-f653d2d07abd,danonear,38552,Price,default,internal,752328,NaN,majority,None,1525.5,0.0,QAT,QAT,1,NaN,1,0,0,0,1,0,0,1,0,0,0
437839,2021-11-03 05:46:28.618424+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,6b58be2b-2a56-45aa-b533-22b2be91d6b4,voting,f6866b2f-0645-432b-b780-f653d2d07abd,danonear,38552,Price,default,internal,752327,NaN,majority,None,1525.5,0.0,QAT,QAT,1,NaN,1,0,0,0,1,0,0,1,0,0,0
437840,2021-11-03 05:46:28.618424+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,6b58be2b-2a56-45aa-b533-22b2be91d6b4,voting,f6866b2f-0645-432b-b780-f653d2d07abd,danonear,38552,Price,default,internal,752326,NaN,majority,None,1525.5,0.0,QAT,QAT,1,NaN,1,0,0,0,1,0,0,1,0,0,0


In [50]:
df.TP_linkage.sum()

27740

In [44]:
t0=df.groupby(['majority_is_promotion','majority_product_fk']).agg({'masking_box':'count'}).reset_index()

In [48]:
t3 = df[(df['majority_is_promotion']==1)]
t3.probe_id.count()

145001

In [49]:
t3.TP_linkage.sum()

6033

In [51]:
t3.TP_promotion.sum()

129324

In [46]:
t1 = df[(df['majority_is_promotion']==1)&(df['majority_product_fk'].isnull())]
t1.probe_id.count()

139215

In [33]:
t2 = df[(df['majority_is_promotion']==1)] 
t2.head()

,timestamp,event_name,session_uid,wave_type,wave_uid,project_name,probe_id,data_subject,flavor,accuracy_logic_type,masking_box,majority_product_fk,voting_result,voting_result_linkage,majority_value,majority_is_promotion,production_tag_source_group,production_tag_source,production_tag_identified,production_product_fk,TP_value,FP_value,FN_value,INCONCLUSIVE_value,TP_promotion,FP_promotion,FN_promotion,TP_linkage,FP_linkage,FN_linkage,INCONCLUSIVE_linkage
3,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086177,NaN,majority,inconclusive,1116.0,1.0,QAT,QAT,1,29.0,1,0,0,0,1,0,0,0,0,0,1
4,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086180,NaN,majority,inconclusive,999.0,1.0,QAT,QAT,1,2142.0,1,0,0,0,1,0,0,0,0,0,1
8,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086173,NaN,majority,inconclusive,799.0,1.0,Engine,Engine,1,567.0,1,0,0,0,1,0,0,0,0,0,1
9,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086174,NaN,majority,inconclusive,889.0,1.0,Engine,Engine,1,882.0,1,0,0,0,1,0,0,0,0,0,1
11,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086176,NaN,majority,inconclusive,729.0,1.0,Engine,Engine,1,30.0,1,0,0,0,1,0,0,0,0,0,1


In [35]:
df[(df['probe_id']==8731043) & (df['project_name']=='diageoru')]

,timestamp,event_name,session_uid,wave_type,wave_uid,project_name,probe_id,data_subject,flavor,accuracy_logic_type,masking_box,majority_product_fk,voting_result,voting_result_linkage,majority_value,majority_is_promotion,production_tag_source_group,production_tag_source,production_tag_identified,production_product_fk,TP_value,FP_value,FN_value,INCONCLUSIVE_value,TP_promotion,FP_promotion,FN_promotion,TP_linkage,FP_linkage,FN_linkage,INCONCLUSIVE_linkage
0,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086182,NaN,inconclusive,inconclusive,NaN,NaN,tag_source_not_relevant,tag_source_not_relevant,0,NaN,0,0,0,1,0,0,0,0,0,0,1
1,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086181,NaN,inconclusive,inconclusive,NaN,NaN,QAT,QAT,1,821.0,0,0,0,1,0,0,0,0,0,0,1
2,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086205,NaN,inconclusive,inconclusive,NaN,NaN,tag_source_not_relevant,tag_source_not_relevant,0,NaN,0,0,0,1,0,0,0,0,0,0,1
3,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086177,NaN,majority,inconclusive,1116.00,1.0,QAT,QAT,1,29.0,1,0,0,0,1,0,0,0,0,0,1
4,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086180,NaN,majority,inconclusive,999.00,1.0,QAT,QAT,1,2142.0,1,0,0,0,1,0,0,0,0,0,1
5,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086206,NaN,majority,inconclusive,2342.00,0.0,QAT,QAT,1,481.0,1,0,0,0,1,0,0,0,0,0,1
6,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086213,NaN,majority,inconclusive,1217.00,0.0,QAT,QAT,1,437.0,1,0,0,0,1,0,0,0,0,0,1
7,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086172,NaN,majority,inconclusive,1107.99,0.0,Engine,Engine,1,881.0,1,0,0,0,1,0,0,0,0,0,1
8,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086173,NaN,majority,inconclusive,799.00,1.0,Engine,Engine,1,567.0,1,0,0,0,1,0,0,0,0,0,1
9,2021-11-23 20:10:22.831549+00:00,PROBE_ACCURACY-DATA-SUMMARIZED,70534a02-fd35-4087-bcdd-4d234f219de0,pricing_voting,6423d92c-9235-4927-b11f-8dea4e71170a,diageoru,8731043,Price,default,internal,8086174,NaN,majority,inconclusive,889.00,1.0,Engine,Engine,1,882.0,1,0,0,0,1,0,0,0,0,0,1


In [ ]:
df_agg = df.['wave_type','',]

In [19]:
sql = """ select *
from trax-ortal-prod.thelake.rds_product
"""
rds_product = bq_client.run_query(sql).to_dataframe()

2021-11-24 15:12:53,254 - Diageo Accuracy Test - 1515 - INFO - Run query with job_id=d0ff07e9-9e58-406a-b09f-084fc0f3f054 
2021-11-24 15:12:54,269 - Diageo Accuracy Test - 1515 - INFO - Query job=d0ff07e9-9e58-406a-b09f-084fc0f3f054 finished {'total_mb_billed': 0.0, 'total_mb_processed': 0.0}


In [20]:
rds_product.head()

,product_pk,product_uuid,product_name,product_local_name,product_type,ean_code,item_code,form_factor,size,unit_of_measure,unit_num,subpackages_num,activation_update_time,creation_date,delete_date,product_width,product_height,product_depth,sku_global_code,is_traxlive,brand_pk,brand_name,brand_local_name,brand_creation_date,brand_delete_date,brand_uuid,brand_global_code,brand_recognition_level_fk,manufacture_name,is_project_manufacturer,brand_recognition_level,category_pk,category_name,category_local_name,category_uuid,trax_category_pk,trax_category_name,trax_category_segment,is_active,manufacturer_pk,trax_category_uuid,labels,substitution_product_fk,trax_palette_level_1_fk,trax_palette_level_1_value,trax_palette_level_2_fk,trax_palette_level_2_value,sub_category_pk,sub_category_name,project_name,_snapshot_date,cloud
0,0,00000000-0000-0000-0000-000000000000,General Empty,General Empty,Empty,None,None,None,NaN,None,NaN,NaN,NaT,2017-12-17 08:40:07+00:00,NaT,NaN,NaN,NaN,None,0,0,General,General,2018-04-08 13:31:30+00:00,NaT,00000000-0000-0000-0000-000000000000,None,1,General,True,Brand,0,General,General,00000000-0000-0000-0000-000000000000,None,None,None,0,0.0,None,None,NaN,[],[],[],[],NaN,None,beiersdorfpe,2021-11-24,AWS
1,9440,93cc2d40-ccb5-11ea-900c-3f85ef381ba9,Irrelevant,Irrelevant,Irrelevant,None,None,None,NaN,None,NaN,NaN,2021-07-21 18:24:13+00:00,2021-07-21 16:56:16+00:00,NaT,NaN,NaN,NaN,None,0,0,General,General,2018-04-08 13:31:30+00:00,NaT,00000000-0000-0000-0000-000000000000,None,1,General,True,Brand,0,General,General,00000000-0000-0000-0000-000000000000,65.0,General,General,1,0.0,e2b80fe0-22c8-4c17-8067-c421a23ce301,{},NaN,[73],[],[2239],"[""General""]",NaN,None,albau,2021-11-24,AWS
2,266,72958910-8000-11eb-a746-853ca5a6627e,Shaving Gel & Foam Other,Shaving Gel & Foam Other,Other,None,None,None,NaN,None,NaN,NaN,2021-03-08 11:21:53+00:00,2021-03-08 11:21:37+00:00,NaT,NaN,NaN,NaN,None,0,0,General,General,2018-04-08 13:31:30+00:00,NaT,00000000-0000-0000-0000-000000000000,None,1,General,True,Brand,0,General,General,00000000-0000-0000-0000-000000000000,186.0,Shaving Gel & Foam,Beauty & Health,1,0.0,0a51898b-8b5f-11e9-ab31-42010a57012a,{},NaN,[10],[],[5],"[""General""]",NaN,None,beiersdorfeg,2021-11-24,AWS
3,628,2adeb0f0-b3d7-11eb-a280-a77ee64a2de4,APDO Other,APDO Other,Other,None,None,None,NaN,None,NaN,NaN,2021-05-13 10:37:15+00:00,2021-05-13 10:37:08+00:00,NaT,NaN,NaN,NaN,None,0,0,General,General,2018-04-08 13:31:30+00:00,NaT,00000000-0000-0000-0000-000000000000,None,1,General,True,Brand,0,General,General,00000000-0000-0000-0000-000000000000,174.0,APDO,Beauty & Health,1,0.0,0a517626-8b5f-11e9-ab31-42010a57012a,{},NaN,[11],[],[7],"[""General""]",NaN,None,beiersdorfkz,2021-11-24,AWS
4,5,3c7a5403-5ed4-11e6-98f2-02af95df48fd,Empty,Empty,Empty,Empty 0000,None,None,NaN,None,NaN,NaN,2018-12-20 10:42:48+00:00,2016-08-10 08:27:12+00:00,2019-01-01 17:56:21+00:00,NaN,NaN,NaN,None,0,0,General,General,2018-04-08 13:31:34+00:00,NaT,00000000-0000-0000-0000-000000000000,None,1,General,True,Brand,0,General,General,00000000-0000-0000-0000-000000000000,None,None,None,0,0.0,None,{},NaN,[],[],[],[],NaN,None,bimy,2021-11-24,AWS


In [30]:
rds_product[(rds_product['project_name']=='danonear')&(rds_product['product_pk']==349.0)] #5510

,product_pk,product_uuid,product_name,product_local_name,product_type,ean_code,item_code,form_factor,size,unit_of_measure,unit_num,subpackages_num,activation_update_time,creation_date,delete_date,product_width,product_height,product_depth,sku_global_code,is_traxlive,brand_pk,brand_name,brand_local_name,brand_creation_date,brand_delete_date,brand_uuid,brand_global_code,brand_recognition_level_fk,manufacture_name,is_project_manufacturer,brand_recognition_level,category_pk,category_name,category_local_name,category_uuid,trax_category_pk,trax_category_name,trax_category_segment,is_active,manufacturer_pk,trax_category_uuid,labels,substitution_product_fk,trax_palette_level_1_fk,trax_palette_level_1_value,trax_palette_level_2_fk,trax_palette_level_2_value,sub_category_pk,sub_category_name,project_name,_snapshot_date,cloud
332781,349,54105220-64eb-11eb-ad87-97ddbbd94778,Nutrilon Pro Futura Liquid Milk Formula From 2...,Nutrilon 4 Profutura Brik X 200 Ml,SKU,7795323002499,None,tetra,200.0,ml,NaN,NaN,2021-02-02 06:37:01+00:00,2021-02-02 00:12:25+00:00,NaT,48.0,120.0,38.0,None,0,57,Nutrilon,Nutrilon,2021-02-02 00:12:24+00:00,NaT,53b33e00-64eb-11eb-ad87-97ddbbd94778,None,2,Nutricia - Bago,True,Product,5,Early Life Nutrition,Early Life Nutrition,51d24900-64eb-11eb-ad87-97ddbbd94778,32.0,Baby & Toddler Food,Baby & Kids,1,17.0,062b3d49-6fa5-4220-9738-afae5d1b7f0d,"{""alt_code_1"": ""Leche"", ""alt_code_2"": ""Liquido...",NaN,[3],"[""Baby Food & Milk""]",[35],[],12.0,Formulas Infantiles,danonear,2021-11-24,AWS


In [22]:
sql_reg = """select * 
        from
        (select
                event_name as recognition_event_name,
                timestamp as recognition_event_timestamp, 
                wave_type as recognition_wave_type,
                project_name,
                probe_id,
                probe_creation_time, 
                data_subject,
                if(data_subject = 'Price', true, false) as is_price_recognition_probe,
                e.tag_source_group,    
                e.tag_source,
                e.product_fk as product_pk,
                e.facings,--not same with sku level
                e.total_price_tags,
                e.price_value,
                e.is_promotion,
                dense_rank() over(partition by event_name, project_name, probe_id, e.tag_source_group order by timestamp desc) as rownum
            from `trax-ortal-prod.raw.factory_recognition`
            cross join unnest(additional_event_data) as e
            where 1=1
                and date(timestamp) > '2021-10-01'
                and wave_type in ('primary' , 'offline_pricing', 'category_expert')
                and e.tag_source_group in ('Engine', 'QAT') 
                and data_subject = 'Price'
                and flavor = 'default'
        ) 
        where 1=1
        and rownum = 1"""

In [23]:
df_recognition = bq_client.run_query(sql_reg).to_dataframe()

2021-11-24 15:13:35,040 - Diageo Accuracy Test - 1515 - INFO - Run query with job_id=49f0eddb-5cc2-4fbf-adbf-c77c8009e5d3 
2021-11-24 15:13:59,502 - Diageo Accuracy Test - 1515 - INFO - Query job=49f0eddb-5cc2-4fbf-adbf-c77c8009e5d3 finished {'total_mb_billed': 24923.602944, 'total_mb_processed': 24923.203034}


In [24]:
df_recognition.head()

,recognition_event_name,recognition_event_timestamp,recognition_wave_type,project_name,probe_id,probe_creation_time,data_subject,is_price_recognition_probe,tag_source_group,tag_source,product_pk,facings,total_price_tags,price_value,is_promotion,rownum
0,PROBE_RECOGNITION-DATA-SUMMARIZED,2021-10-15 03:27:34.543687+00:00,offline_pricing,diageoke,10076871,2021-10-14 13:51:28+00:00,Price,True,QAT,QAT,201.0,1,1,2300.0,None,1
1,PROBE_RECOGNITION-DATA-SUMMARIZED,2021-10-27 12:57:17.754145+00:00,offline_pricing,diageoke,10207479,2021-10-26 14:58:03+00:00,Price,True,QAT,QAT,124.0,1,1,4962.0,None,1
2,PROBE_RECOGNITION-DATA-SUMMARIZED,2021-10-28 14:06:45.063028+00:00,offline_pricing,diageoke,10232957,2021-10-27 15:59:18+00:00,Price,True,QAT,QAT,1122.0,1,1,7000.0,None,1
3,PROBE_RECOGNITION-DATA-SUMMARIZED,2021-11-17 07:09:32.669906+00:00,offline_pricing,diageoke,10468763,2021-11-16 16:59:06+00:00,Price,True,QAT,QAT,349.0,1,1,330.0,None,1
4,PROBE_RECOGNITION-DATA-SUMMARIZED,2021-11-03 16:41:33.862641+00:00,offline_pricing,diageogh,8004797,2021-11-03 14:58:25+00:00,Price,True,QAT,QAT,NaN,1,1,32.0,None,1


In [25]:
df_recognition[(df_recognition['probe_id']==664187) & (df_recognition['project_name']=='jnjanz')&(df_recognition['product_pk']==5510.0)]

,recognition_event_name,recognition_event_timestamp,recognition_wave_type,project_name,probe_id,probe_creation_time,data_subject,is_price_recognition_probe,tag_source_group,tag_source,product_pk,facings,total_price_tags,price_value,is_promotion,rownum
64877134,PROBE_RECOGNITION-DATA-SUMMARIZED,2021-10-29 10:04:28.575145+00:00,offline_pricing,jnjanz,664187,2021-10-28 23:29:15+00:00,Price,True,Engine,Engine,5510.0,1,1,48.99,None,1
